# Comparison between pyBKT and BKT with pyAgrum
In this notebook, we want to make sure that the way we implement BKT with pyAgrum corresponds to the values obtained by pyBKT. 

In [1]:
import sys
sys.path.append("/Users/olivier/PycharmProjects/bayesian-kst/")  # for mac

import pyAgrum as gum
from kgraph.expert_layer.domain import Domain
from kgraph.expert_layer.knowledge_components import KnowledgeComponent
from kgraph.expert_layer.link import Link
from kgraph.resources_layer.exercise import Exercise
from kgraph.learner_layer.answer import LearnerAnswer
from kgraph.learner_layer.learner import Learner
from kgraph.learner_layer.learner_pool import LearnerPool
from kgraph.helpers.truthtable import truthtable
from math import floor
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.dynamicBN as gdyn
import random
import itertools
import numpy as np
import tqdm

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold

def get_strongest_folds(full, axis="user_id", nb_folds=5):
    all_elements = full[axis].unique()

    kfold = KFold(nb_folds, shuffle=True)
    folds = []
    for i, (train, test) in enumerate(kfold.split(all_elements)):
        list_of_test_ids = []
        for element_id in test:
            list_of_test_ids += list(full.query(f'{axis} == {all_elements[element_id]}').index)
        folds.append(np.array(list_of_test_ids))
    
    return folds

def mae(true_vals, pred_vals):
    """ Calculates the mean absolute error. """
    return np.mean(np.abs(true_vals - pred_vals))


## Define the domain knowledge model

In [2]:

# we define the KCs

KC_A = KnowledgeComponent(55365, "Déterminer l'appartenance d'un nombre réel à un intervalle fini")
KC_B = KnowledgeComponent(55363, "Déterminer l'appartenance d'un nombre réel à un intervalle infini")
KC_C = KnowledgeComponent(55364, "Déterminer l'appartenance d'un nombre réel à un intervalle simple")
KC_D = KnowledgeComponent(50988, "Déterminer l'appartenance d'un nombre réel à une intersection d'intervalles de R")
KC_E = KnowledgeComponent(50989, "Déterminer l'appartenance d'un nombre réel à une réunion d'intervalles de R")

def get_KC_from_exercise_id(exercise_id):
    if exercise_id in range(237957, 237962):
        return KC_A
    elif exercise_id in range(237947, 237952):
        return KC_B
    elif exercise_id in range(237952, 237957):
        return KC_C
    elif exercise_id in range(225183, 225188):
        return KC_D
    else:
        return KC_E
    

## Import the data

In [3]:
import pandas as pd

df = pd.read_csv("5_KCs_example_data.csv")

kc_ids = []
for i, row in df.iterrows():
    kc_ids.append(get_KC_from_exercise_id(row['exercise_id']).id)
    
df['kc_id'] = kc_ids

print(df)

folds = get_strongest_folds(df, "user_id", 2)
test_ids = folds[0]

train_ids = list(set(list(df.index.values)) - set(test_ids))

df_train = df[df.index.isin(train_ids)]
df_test = df[df.index.isin(test_ids)]

       exercise_id  evaluation_id  success  user_id            createdAt  \
0           225183      109276367        0   757204  2019-08-12 12:57:49   
1           225183      109293461        1  2052585  2019-08-12 19:18:03   
2           225183      109293517        1  2052585  2019-08-12 19:20:45   
3           225183      109293574        1  2052585  2019-08-12 19:23:40   
4           225183      109307385        1  1896564  2019-08-13 11:16:49   
...            ...            ...      ...      ...                  ...   
42006       237961      151532622        1  3940614  2021-09-08 12:49:53   
42007       237961      151546756        1  1970804  2021-09-08 23:48:52   
42008       237961      151546760        1  1970804  2021-09-08 23:49:21   
42009       237961      151549675        1  3943368  2021-09-09 11:07:45   
42010       237961      151549982        0  3940672  2021-09-09 11:22:25   

       kc_id  
0      50988  
1      50988  
2      50988  
3      50988  
4      50988

In [4]:
from kgraph.helpers.import_dataset import *
defaults = {'learner_id': 'user_id', 'kc_id': 'kc_id', 'exercise_id': 'exercise_id', 'success':'success'}
domain, exercises = setup_domain_and_resources_from_dataset(df_test, defaults)
print(domain)

Domain on 5 KCs:
- KC 50988: 50988
- KC 50989: 50989
- KC 55363: 55363
- KC 55364: 55364
- KC 55365: 55365



## Learn parameters with pyBKT
We learn the parameters of the bayesian network with pyBKT (EM algorithm) on the train dataset. 

In [5]:
from pyBKT.models import Model

# Initialize the model with an optional seed
model = Model(seed = 42, num_fits = 1)
pybkt_defaults = {'order_id': 'evaluation_id',
            'skill_name': 'kc_id',
            'correct': 'success',
            'user_id': 'user_id',
            'multigs': 'exercise_id',
            'folds': 'user_id'
           }

model.fit(data = df_train, defaults = pybkt_defaults, multigs = True)


def set_learning_pool_parameters_from_bkt_parameters(learner_pool, bkt_params):
    for kc in learner_pool.get_knowledge_components():
        learner_pool.set_prior(kc, model.params().loc[f'{kc.id}', 'prior', 'default'].value)
        
        learner_pool.set_learn(kc, model.params().loc[f'{kc.id}', 'learns', 'default'].value)
        learner_pool.set_forget(kc, model.params().loc[f'{kc.id}', 'forgets', 'default'].value)
        
        for exercise in kc.get_exercises():
            learner_pool.set_guess(exercise, model.params().loc[f'{kc.id}', 'guesses', f'{exercise.id}'].value)
            learner_pool.set_slip(exercise, model.params().loc[f'{kc.id}', 'slips', f'{exercise.id}'].value)
    return learner_pool

params = model.params()
print(params)
learner_pool = LearnerPool(domain, {})
learner_pool = set_learning_pool_parameters_from_bkt_parameters(learner_pool, params)



                        value
skill param   class          
50988 prior   default 0.18099
      learns  default 0.22086
      guesses 225183  0.15588
              225184  0.07036
              225185  0.20044
...                       ...
55364 slips   237953  0.01315
              237954  0.04170
              237955  0.02209
              237956  0.05330
      forgets default 0.00000

[65 rows x 1 columns]


In [6]:
print(len(df_test.index))
df_test.head()

20554


,exercise_id,evaluation_id,success,user_id,createdAt,kc_id
0,225183,109276367,0,757204,2019-08-12 12:57:49,50988
4,225183,109307385,1,1896564,2019-08-13 11:16:49,50988
5,225183,109320794,1,2623038,2019-08-13 15:41:35,50988
18,225183,109636965,1,1578689,2019-08-21 11:19:01,50988
21,225183,109648375,0,2015032,2019-08-21 14:14:01,50988


## Check prediction performance of pyBKT model
We then check how the prediction is performed with pyBKT and the learned parameters.

In [7]:
print(model.predict(data=df_test))

print('ACC', model.evaluate(data = df_test, metric = 'accuracy'))
print('AUC', model.evaluate(data = df_test, metric = 'auc'))
print('MAE', model.evaluate(data = df_test, metric = mae))

       exercise_id  evaluation_id  success  user_id            createdAt  \
11471       225186      116645380        0    76926  2019-12-15 19:37:52   
1320        225183      116645707        0    76926  2019-12-15 19:40:44   
24346       225168      116645842        0    76926  2019-12-15 19:41:50   
4247        225184      110931718        0   119344  2019-09-20 10:35:07   
10692       225186      110931729        0   119344  2019-09-20 10:35:25   
...            ...            ...      ...      ...                  ...   
19001       225165      151549833        1  3940614  2021-09-09 11:13:37   
23628       225167      151549840        0  3940614  2021-09-09 11:14:03   
21396       225166      151549853        1  3940614  2021-09-09 11:14:47   
27889       225169      151549867        1  3940614  2021-09-09 11:15:24   
25838       225168      151549873        1  3940614  2021-09-09 11:16:07   

       kc_id  correct_predictions  state_predictions  
11471  50988              0.2488

## Check prediction performance of BKT with pyAgrum library
We setup the same process for BKT with pyAgrum, with parameters computed thanks to pyBKT. 

In [8]:
def get_bkt_net(kc, learner, learner_traces):
    bkt_net = gum.BayesNet('BKT')
    
    learn = learner.learner_pool.get_learn(kc)
    prior = learner.learner_pool.get_prior(kc)
    forget = learner.learner_pool.get_forget(kc)

    bkt_net.add(gum.LabelizedVariable(f"({kc.name})0", '', 2))
    bkt_net.cpt(f"({kc.name})0").fillWith([1-prior, prior])

    bkt_net.add(gum.LabelizedVariable(f"({kc.name})t", '', 2))
    for exercise in kc.get_exercises():
        bkt_net.add(gum.LabelizedVariable(f"exercise({exercise.id})0", '', 2))
        bkt_net.addArc(*(f"({kc.name})0", f"exercise({exercise.id})0"))

        bkt_net.add(gum.LabelizedVariable(f"exercise({exercise.id})t", '', 2))
        bkt_net.addArc(*(f"({kc.name})t", f"exercise({exercise.id})t"))

        guess = learner.learner_pool.get_guess(exercise)
        slip = learner.learner_pool.get_slip(exercise)
        bkt_net.cpt(f"exercise({exercise.id})0")[{f"({kc.name})0": False}] = [1-guess, guess]
        bkt_net.cpt(f"exercise({exercise.id})0")[{f"({kc.name})0": True}] = [slip, 1-slip]

        bkt_net.cpt(f"exercise({exercise.id})t")[{f"({kc.name})t": False}] = [1-guess, guess]
        bkt_net.cpt(f"exercise({exercise.id})t")[{f"({kc.name})t": True}] = [slip, 1-slip]

    bkt_net.addArc(*(f"({kc.name})0", f"({kc.name})t"))
    bkt_net.cpt(f"({kc.name})t")[{f"({kc.name})0": False}] = [1-learn, learn]
    bkt_net.cpt(f"({kc.name})t")[{f"({kc.name})0": True}] = [forget, 1-forget]

    n_eval = len(learner_traces)
    return gdyn.unroll2TBN(bkt_net, n_eval+1)



def get_bkt_net_bis(kc, learner, learner_traces):
    bkt_net = gum.BayesNet('BKT')
    
    learn = learner.learner_pool.get_learn(kc)
    prior = learner.learner_pool.get_prior(kc)
    forget = learner.learner_pool.get_forget(kc)

    bkt_net.add(gum.LabelizedVariable(f"({kc.name})0", '', 2))
    bkt_net.cpt(f"({kc.name})0").fillWith([1-prior, prior])

    bkt_net.addAND(gum.LabelizedVariable(f"({kc.name})t", '', 2))
    for exercise in kc.get_exercises():
        bkt_net.add(gum.LabelizedVariable(f"exercise({exercise.id})0", '', 2))
        bkt_net.addArc(*(f"({kc.name})0", f"exercise({exercise.id})0"))

        bkt_net.add(gum.LabelizedVariable(f"exercise({exercise.id})t", '', 2))
        bkt_net.addArc(*(f"({kc.name})t", f"exercise({exercise.id})t"))

        guess = learner.learner_pool.get_guess(exercise)
        slip = learner.learner_pool.get_slip(exercise)
        bkt_net.cpt(f"exercise({exercise.id})0")[{f"({kc.name})0": False}] = [1-guess, guess]
        bkt_net.cpt(f"exercise({exercise.id})0")[{f"({kc.name})0": True}] = [slip, 1-slip]

        bkt_net.cpt(f"exercise({exercise.id})t")[{f"({kc.name})t": False}] = [1-guess, guess]
        bkt_net.cpt(f"exercise({exercise.id})t")[{f"({kc.name})t": True}] = [slip, 1-slip]

        
    bkt_net.add(gum.LabelizedVariable(f"Z[({kc.name})_0 -> ({kc.name})_t]t", '', 2))

    bkt_net.addArc(*(f"({kc.name})0", f"Z[({kc.name})_0 -> ({kc.name})_t]t"))
    bkt_net.addArc(*(f"Z[({kc.name})_0 -> ({kc.name})_t]t", f"({kc.name})t"))

    bkt_net.cpt(f"Z[({kc.name})_0 -> ({kc.name})_t]t")[{f"({kc.name})0": False}] = [1-learn, learn]
    bkt_net.cpt(f"Z[({kc.name})_0 -> ({kc.name})_t]t")[{f"({kc.name})0": True}] = [forget, 1-forget]

    n_eval = len(learner_traces)
    return gdyn.unroll2TBN(bkt_net, n_eval+1)


def evaluate_learner_bkt(learner_traces):
    n_eval = len(learner_traces)
    evaluated_kc = learner_traces[0].get_kc()
    learner = learner_traces[0].get_learner()
    
    floor_idx = 0
    expected_values = []
    predicted_values = []
    
    for i in range(len(learner_traces)):

        bn = get_bkt_net(evaluated_kc, learner, learner_traces[:i])

        ie=gum.LazyPropagation(bn)

        if i > 0:
            ie.setEvidence({
                f"exercise({learner_traces[j].get_exercise().id}){j}": learner_traces[j].get_success() for j in range(i)
            })
        ie.makeInference()
        predicted_values.append(ie.posterior(f"exercise({learner_traces[i].get_exercise().id}){i}")[:][1])
        expected_values.append(learner_traces[i].get_success())
        
    return (expected_values, predicted_values)

def evaluate_learner_bkt_bis(learner_traces):
    n_eval = len(learner_traces)
    evaluated_kc = learner_traces[0].get_kc()
    learner = learner_traces[0].get_learner()
    
    floor_idx = 0
    expected_values = []
    predicted_values = []
    
    for i in range(len(learner_traces)):

        bn = get_bkt_net_bis(evaluated_kc, learner, learner_traces[:i])

        ie=gum.LazyPropagation(bn)

        if i > 0:
            ie.setEvidence({
                f"exercise({learner_traces[j].get_exercise().id}){j}": learner_traces[j].get_success() for j in range(i)
            })
        ie.makeInference()
        predicted_values.append(ie.posterior(f"exercise({learner_traces[i].get_exercise().id}){i}")[:][1])
        expected_values.append(learner_traces[i].get_success())
        
    return (expected_values, predicted_values)


In [9]:
### Score with pyAgrum

learner_traces = deduce_learner_traces_from_dataset(df_test, exercises, learner_pool, defaults)

expected_values = []
predicted_values = []


for learner in tqdm.tqdm(list(learner_traces.keys())):
    for kc in list({object_.id: object_ for object_ in [trace.get_kc() for trace in learner_traces[learner]]}.values()):
        kc_learner_evals = [trace for trace in learner_traces[learner] if trace.get_kc() is kc]
        floor_idx = 0
        if kc_learner_evals:
            exp_vals, pred_vals = evaluate_learner_bkt(kc_learner_evals)
            predicted_values = np.concatenate((predicted_values, pred_vals))
            expected_values = np.concatenate((expected_values, exp_vals))

print('ACC', accuracy_score(expected_values, [1 if x>.5 else 0 for x in predicted_values]))
print('AUC', roc_auc_score(expected_values, predicted_values))
print('MAE', mae(expected_values, predicted_values))


100%|██████████| 1107/1107 [04:38<00:00,  3.98it/s]

ACC 0.7652038532645714
AUC 0.7687374177840008
MAE 0.3262661275149369


In [10]:
### Score with pyAgrum

learner_traces = deduce_learner_traces_from_dataset(df_test, exercises, learner_pool, defaults)

expected_values = []
predicted_values = []


for learner in tqdm.tqdm(list(learner_traces.keys())):
    for kc in list({object_.id: object_ for object_ in [trace.get_kc() for trace in learner_traces[learner]]}.values()):
        kc_learner_evals = [trace for trace in learner_traces[learner] if trace.get_kc() is kc]
        floor_idx = 0
        if kc_learner_evals:
            exp_vals, pred_vals = evaluate_learner_bkt_bis(kc_learner_evals)
            predicted_values = np.concatenate((predicted_values, pred_vals))
            expected_values = np.concatenate((expected_values, exp_vals))

print(predicted_values)
print('ACC', accuracy_score(expected_values, [1 if x>.5 else 0 for x in predicted_values]))
print('AUC', roc_auc_score(expected_values, predicted_values))
print('MAE', mae(expected_values, predicted_values))


100%|██████████| 1107/1107 [04:53<00:00,  3.77it/s]

[0.24882399 0.34124782 0.42781673 ... 0.44242548 0.75064631 0.66635083]
ACC 0.7652038532645714
AUC 0.7687374177840008
MAE 0.3262661275149369


Conclusion: same results but a way longer compute time

In [11]:
predicted_values, expected_values = [], []
for learner in tqdm.tqdm(list(learner_traces.keys())):
    expected_values = np.concatenate((expected_values,
                                      [int(trace.get_success()) for trace in learner_traces[learner]]))
    predicted_values = np.concatenate((predicted_values, 
                                       learner.predict_sequence(learner_traces[learner], {}, {})))
print(predicted_values, expected_values)
print('ACC', accuracy_score(expected_values, [1 if x>.5 else 0 for x in predicted_values]))
print('AUC', roc_auc_score(expected_values, predicted_values))
print('MAE', mae(expected_values, predicted_values))

100%|██████████| 1107/1107 [23:36<00:00,  1.28s/it] 

[0.24882399 0.34871695 0.42781673 ... 0.43156513 0.75450268 0.6668405 ] [0. 0. 0. ... 1. 1. 1.]
ACC 0.7666634231779702
AUC 0.7462365358486472
MAE 0.3473110640592006
